In [1]:
%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588603162430_0001,pyspark,idle,Link,Link,


In [3]:
sc.install_pypi_package("pandas==1.0.3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [4]:
sc.install_pypi_package("matplotlib==3.2.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)

In [50]:
user=spark.read.json('s3://sta9760-laura-spark-datasets/yelp-dataset/yelp_academic_dataset_user.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
print(f'Total Columns: {len(user.dtypes)}')
print(f'Total Rows: {user.count():,}')
user.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 22
Total Rows: 1,968,703
root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

In [55]:
business=spark.read.json('s3://sta9760-laura-spark-datasets/yelp-dataset/yelp_academic_dataset_business.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import col
#user.select('business_id','stars').withColumn('stars', col("stars").cast(DecimalType(25,16))).show(5)
business.select('business_id','stars').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|         business_id|stars|
+--------------------+-----+
|f9NumwFMBDn751xgF...|  3.5|
|Yzvjg0SayhoZgCljU...|  5.0|
|XNoUzKckATkOD1hP6...|  5.0|
|6OAZjbxqM5ol29BuH...|  2.5|
|51M2Kk903DFYI6gnB...|  4.5|
+--------------------+-----+
only showing top 5 rows

In [58]:
business_avg=business.groupby('business_id').agg({'stars':'mean'})
#user_avg=user.groupby('business_id').avg('stars')
business_avg.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|         business_id|avg(stars)|
+--------------------+----------+
|M_guz7Dj7hX0evS67...|       3.5|
|oqbNgOBAxsa49c2WQ...|       3.5|
|2BKKUjLSocdlgfsMC...|       5.0|
|DloBjCZHlz0lzsrQJ...|       4.0|
|lj0MiK5_fyv9df2tw...|       4.5|
+--------------------+----------+
only showing top 5 rows

In [61]:
business_and_avg = business.join(business_avg, on="business_id", how="inner")
#select("user.stars","business.name","business.city","business.state")
business_and_avg.select('name','city','state','stars','avg(stars)').show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+-----+-----+----------+
|                name|           city|state|stars|avg(stars)|
+--------------------+---------------+-----+-----+----------+
|The Range At Lake...|      Cornelius|   NC|  3.5|       3.5|
|   Carlos Santo, NMD|     Scottsdale|   AZ|  5.0|       5.0|
|             Felinus|       Montreal|   QC|  5.0|       5.0|
|Nevada House of Hose|North Las Vegas|   NV|  2.5|       2.5|
|USE MY GUY SERVIC...|           Mesa|   AZ|  4.5|       4.5|
|Oasis Auto Center...|        Gilbert|   AZ|  4.5|       4.5|
|Green World Cleaners|      Las Vegas|   NV|  3.5|       3.5|
|Junction Tire & A...|           Mesa|   AZ|  5.0|       5.0|
| The Empanadas House|      Champaign|   IL|  4.5|       4.5|
|      Xtreme Couture|        Toronto|   ON|  3.0|       3.0|
|Chinook Landscapi...|        Calgary|   AB|  5.0|       5.0|
|Dependable Brakes...|     Pittsburgh|   PA|  5.0|       5.0|
|Chocolate Shoppe ...|      Middleton|   WI|  3.5|       3.5|
|       